# Flipkart Mobile Details Scraping

- Enter number of page to scrap and Brand name
- URL is dynamic (Only for mobiles of some brand availabe in flipkart mibile section)

- It will auto scrap the data then clean and save it for you in .CSV/.xlsx format.

[Shorten Link Article](https://towardsdatascience.com/best-apis-for-url-shortening-using-python-2db09d1f86f0)

>  **Brand Name**
- Mi, Realme, Samsung, Infinix, Apple, OPPO, Vivo

Author @ Krishn Kant Raj

Dated: 14/05/2021

In [ ]:
# !pip install bitly_api

In [ ]:
# import important libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import bitly_api

# Bitly Access Token
BITLY_ACCESS_TOKEN = '749d6a714f1342aaced60e021f4a05ced8c7841d'
access = bitly_api.Connection(access_token = BITLY_ACCESS_TOKEN)

In [ ]:
# Create empty lists
Name = []
Price = []
Rating = []
Review = []
Links = []

# Take input of brand name
brand = input('Enter Mobile Brand Name : ')
# Make it in title format
brand = brand.title().split()
# Take the brand name only
brand = brand[0]
    
try:
#     Dynamic URL
    flipkart_url = f'https://www.flipkart.com/mobiles/pr?sid=tyy%2C4io&otracker=categorytree&p%5B%5D=facets.brand%255B%255D%3D{brand}&otracker=nmenu_sub_Electronics_0_{brand}'.format(brand)
except InvalidSchema as er:
    print(er)
    print('Invalid Input')
    
# Show URL    
print(flipkart_url)

# Take how many page to scrap for given mobile brand 
pg_num = int(input('Enter Numbers of Page to scrap: '))

for i in range(1,pg_num+1):
    url = flipkart_url+"&page="+str(i)
    try:
        req = requests.get(url)
    except ConnectionError as er:
        print(er)
        print('Please check your connection!')
    soup = BeautifulSoup(req.content, 'html.parser')
    name = soup.find_all('div',{"class":"_4rR01T"})
    price = soup.find_all('div',{"class":"_30jeq3 _1_WHN1"})
    ratings_reviews = soup.find_all('span',{"class":"_2_R_DZ"})
    links = soup.find_all('a',{'class':'_1fQZEK'})
    print("[INFO] Phones in Page "+str(i)+' is '+str(len(name)))
    
    if len(name)==0:
        print('Invalid URL! Please check the enered URL.')
        exit(0)

    for i in name:
        Name.append(i.text)
    for i in price:
        Price.append(i.text)
    for i in ratings_reviews:
        Rating.append(i.span.span.text)
        Review.append(i.span.text)
    print('[INFO] Shortening link...')
    for i in links:
        linktext = 'https://www.flipkart.com'+i.get('href')
        short_url = access.shorten(linktext) 
        Links.append(short_url['url'])
    if len(name)<24:
        break
    
print("[INFO] Raw data scraped.")
print('\n***ACKNOWLEDGEMENT***')        
print('Name Count=',len(Name))
print('Price Count=',len(Price))
print('Rating Count=',len(Rating))
print('Review Count=',len(Review))
print('Links Count=',len(Links))

In [ ]:
if (len(Name)==len(Price) and len(Name)==len(Rating) and len(Name)==len(Review) and len(Name)==len(Links)):
    print('Woo hoo! No missing Data found')
if len(Price)<len(Name):
    for i in range(0,(len(Name)-len(Price))):
        Price.append('None')
if len(Rating)<len(Name):
    for i in range(0,(len(Name)-len(Rating))):
        Rating.append('None')
if len(Review)<len(Name):
    for i in range(0,(len(Name)-len(Review))):
        Review.append('None')
if len(Links)<len(Name):
    for i in range(0,(len(Name)-len(Links))):
        Links.append('None')

In [ ]:
#  See missing data filled with None or not
#  Get the length of all lists
print(len(Name))
print(len(Price))
print(len(Review))
print(len(Rating))
print(len(Links))

In [ ]:
Name

In [ ]:
# Remove noise form price and clean it
clean_price = []
for i in range(len(Price)):
    price = Price[i][1:].replace(',','')
    clean_price.append(price)
clean_price

In [ ]:
# Extract Reviews counts
# Clean the data
clean_review = []
for i in range(len(Review)):
    if Review[i]=='None':
        clean_review.append('None')
    else:
        Review_str = Review[i].split('\xa0&\xa0',1)[1]
        Review_str = Review_str.replace(' Reviews','').replace(',','')
        clean_review.append(Review_str)
clean_review

In [ ]:
# Extract Rating counts
# clean the data
clean_rating = []
for i in range(len(Rating)):
    if Rating[i]=='None':
        clean_rating.append('None')
    else:
        Rating_str = Rating[i][:-1]
        Rating_str = Rating_str.replace(' Ratings','').replace(',','')
        clean_rating.append(Rating_str)
clean_rating

In [ ]:
# Create a dictionary and convert to dataframe
data = {
    'Product_Name':Name,
    'Price':clean_price,
    'Rating':clean_rating,
    'Review':clean_review,
    'Link':Links
}
data = pd.DataFrame(data)
data

In [ ]:
# Describe the data
data.describe()

In [ ]:
# If your string constraint is not just one string you can drop those corresponding rows with:
# df = df[~df['your column'].isin(['list of strings'])]

# Remove all rows having 'None' string
data = data[~data.Rating.str.contains("None")]

In [ ]:
# Get the info
data.info()

In [ ]:
# convert all object to numeric form
data[['Price','Rating','Review']] = data[['Price','Rating','Review']].astype(int)

In [ ]:
# get updated info
data.info()

In [ ]:
# Last five rows
data.tail(5)

In [ ]:
# First five rows
data.head(5)

In [ ]:
# Sample Data
# Randomly selected
data.sample(5)

In [ ]:
# Save file with brand name
filename = brand+'.xlsx'   
try:    
    writer = pd.ExcelWriter(filename, engine='xlsxwriter', options={'strings_to_urls': True})
    data.to_excel(writer,index=False)
    print("['INFO'] File saved successfully!")
except PermissionError as pr:
    print(pr)
    print('Close the pre opended file with same file name')
writer.close()